# Penny Stock Model

In [1]:
# import libraries
import pandas as pd
import yfinance as yf
import numpy as np
import datetime

In [2]:
# parameters
VolRise = 0.99  # open position, after current volume will be more than VolRise value
AvgVolDayLimit = 10 # average for shifting average
ExitAfter = 0.25 # close position after price achieve ExitAfter
StopLoss = -0.07 # close position after price meets stop
RiskPertrade = 2000 # buy for sum 
TradeAfterDays = 3 # begin trade after TradeAfterDays days from start
price_filter = 0.5 # below this price we open pos
high_prevClose = 0.1 # if high more than previous close on high_prevClose
UpperBandVol = 0.7      # if price is in range of  UpperBandVol and LowwerBandVol - please take a volume
LowwerBandVol = 0.09    #   if price is in range of  UpperBandVol and LowwerBandVol - please take a volume
positionValue = 10000  # the sum you need to collect





In [3]:
# для класу deposit потрібно
# кількість акцій
# середня ціна
# дата старту набору позиції
# дата закінчення набору позиції
class StockBalance():
    positionValue = 10000
   
    def __init__(self, ticker, start):
        self.ticker = ticker
        self.start = start
        self.market_value = 0
        self.end_date = 0
        self.qty = 0
        self.avg_price = 0 

    
    def add(self,amount, price):
        self.qty += amount
        val=price*amount
        self.market_value += val
        self.avg_price = self.market_value/self.qty

         
    def is_full(self):
        return self.market_value >=StockBalance.positionValue
    
    def clear(self):
        self.qty = 0
        self.market_value = 0
        self.avg_price = 0

        
        
        





In [5]:
stock = StockBalance('Aapl','2022-01-02')

In [8]:
#stock.add(100000)
#stock.health
stock.is_full()

False

In [9]:
stock.add(500,1)

In [77]:
stock.clear()

In [82]:
stock.avg_price

0.75

In [11]:
stock.market_value

1000

In [3]:
def good_price(ticker_name,date_from_list):
    
    # function to calculate appropriate price
    res = False
    stock_ticker = yf.Ticker(ticker_name)
    r = stock_ticker.history(max,"1d",date_from_list)
    price = r.iloc[0]["Close"]
    if price<price_filter:
        res = True

    return res


In [5]:
#reed.history("1mo","1d","2022-10-23","2023-01-10")


In [34]:
# load ticker list from excel-file
list_stocks = pd.read_excel('inputFiles/list_stocks.xlsx', index_col ='Company')
all_res = []
data_group=[]
data_group_res=[]

stock_res =[]
stock_small_res=[]
#len(list_stocks)

for i in range(50): # потім прописати весь список
    symbol = list_stocks.index[i] # беремо акцію по індексу

    datefromlist = list_stocks.iloc[i]['NotDate']
    #print('symbol=',symbol)
    #print('datefromlist=',datefromlist)
    if good_price(symbol,datefromlist)==False: # if price meet conditions
        continue

    # to calc average volume use 20 days before
    start_date = datefromlist - datetime.timedelta(days=19)                                           
    end_date = datefromlist + datetime.timedelta(days=180)

    

    #load stock price
    #symbol = list_stocks.index[i] # беремо акцію по індексу
    wdf = yf.download(symbol, start_date, end_date)



    # building features 
    wdf['avgvol']= wdf['Volume'].rolling(window=AvgVolDayLimit, win_type=None).mean()  # середній обєм за 10 днів
    wdf['VolRise'] =wdf['Volume']/wdf['avgvol']-1      # building vol signal
    wdf['PrevClose'] = wdf.Close.shift(1)      # write previous close
    wdf['volsignal']=wdf['VolRise']>VolRise            # volume signal column
    wdf['GreenCandleSig'] = wdf['High']>wdf['Close']   # green candle
    wdf['HighGrowPrevClose'] = (wdf["High"]/wdf['PrevClose']-1)> high_prevClose # high is bigger than previous low

    wdf['TradeAfter']=wdf.index

    wdf['TradeAfterDaysSig'] = wdf['TradeAfter'] > datefromlist + datetime.timedelta(days=TradeAfterDays)
    wdf['OpenPrice']=0
    wdf['Profit']=0
    wdf['StopPrice']=0
    wdf['TargetPrice']=0
    wdf['Ticker']=symbol
    del(wdf['TradeAfter'])
    lastrow=wdf.tail(1).index # last row in order to stop calculations

    # main module
    PriceOpen = 0
    StopPrice = 0
    TPPrice = 0
    CloseSignal = 0
    status = 0
    
    
    for index, row in wdf.iterrows():  # change to itter....
        # looking for opening signal 
        if (PriceOpen ==0) & (lastrow[0]!=index): # if we haven't opened position and it's not last row
            
            if row['volsignal'] & row['TradeAfterDaysSig'] & row['GreenCandleSig'] & row['HighGrowPrevClose']:  # if Volume > than AvgVolume and green candle and high more then 
                if row["Close"]>price_filter: # if price for opening is bigger than limit - please skip it
                    continue

                # open position
                PriceOpen = row['Close']  # record close price as price of open
                wdf.at[index,'OpenPrice']= PriceOpen
                StopPrice = PriceOpen-(PriceOpen*StopLoss*-1) # setting stop-loss
                TPPrice = PriceOpen+(PriceOpen*ExitAfter)  # set target price
                status=1
                
                # volume size block
                



    #     # working with first opened position
        if (PriceOpen >0) & (status==0):
            wdf.at[index,'OpenPrice']= PriceOpen
            wdf.at[index,'StopPrice']= StopPrice
            wdf.at[index,'TargetPrice']= TPPrice


            if row['High']>TPPrice:
                wdf.at[index,'Profit']= RiskPertrade*ExitAfter
                PriceOpen =0
                TPPrice=0
                StopPrice=0

            if row['Low']<StopPrice:
                wdf.at[index,'Profit']= RiskPertrade*StopLoss
                PriceOpen =0
                TPPrice=0
                StopPrice=0


        status = 0    

        # if there is open position and its last row
        if lastrow[0]==index:
            if PriceOpen >0:
                wdf.at[index,'Profit']=(row['Close']/PriceOpen-1)*RiskPertrade
                PriceOpen =0
                TPPrice=0
                StopPrice=0
   
    
    #тут закінчується блок перебору строк акції і йде на наступну акцію
    # 1. тікер
    # 2. profit
    # 3. tradecount
    # 4. ProfitTrades
    #  5.   LossTrades
    # 6.    WinRate
    #7.     AvgHold'
    
    tradecount=(wdf['Profit']!=0).sum()
    ProfitTrades = (wdf['Profit']>0).sum()
    LossTrades = (wdf['Profit']<0).sum()
    #WinRate = round(ProfitTrades/tradecount,1)
    #AvgHold = round((wdf['OpenPrice']!=0).sum()/tradecount,1)
    stock_small_res=[symbol,round(wdf['Profit'].sum(),2),tradecount,ProfitTrades,LossTrades]
    
    data_group=wdf['Profit']
    stock_res.append(stock_small_res)
    data_group_res.append(data_group)
    # блок запису dataFrame:
    all_res.append(wdf)
      
    df_output = pd.concat(all_res)
    
    df_data =pd.concat(data_group_res)
    #df_data.columns['Profit']
   # mask1 = df_data.Profit >0
    #mask2 = df_data.Profit <0
    #df_out= df_data.loc[df_data['Profit']!=0]



    
      
        

NameError: name 'good_price' is not defined

In [7]:
df_output

,Open,High,Low,Close,Adj Close,Volume,avgvol,VolRise,PrevClose,volsignal,GreenCandleSig,HighGrowPrevClose,TradeAfterDaysSig,OpenPrice,Profit,StopPrice,TargetPrice,Ticker
Date,,,,,,,,,,,,,,,,,,
2022-01-24,0.163029,0.165659,0.152511,0.165659,-0.184924,2872406,NaN,NaN,NaN,False,False,False,False,0.000000,0.000000,0.00000,0.000000,CBIO
2022-01-25,0.178806,0.197213,0.152511,0.155141,-0.173183,24439979,NaN,NaN,0.165659,False,True,True,False,0.000000,0.000000,0.00000,0.000000,CBIO
2022-01-26,0.152511,0.160400,0.147252,0.149882,-0.167312,4451411,NaN,NaN,0.155141,False,True,False,False,0.000000,0.000000,0.00000,0.000000,CBIO
2022-01-27,0.152511,0.157770,0.139364,0.144623,-0.161441,3194900,NaN,NaN,0.149882,False,True,False,False,0.000000,0.000000,0.00000,0.000000,CBIO
2022-01-28,0.144623,0.152511,0.139364,0.144623,-0.161441,1959306,NaN,NaN,0.144623,False,True,False,False,0.000000,0.000000,0.00000,0.000000,CBIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-03,0.468052,0.483829,0.465422,0.478570,-0.534224,2236164,1263888.8,0.769273,0.468052,False,True,False,True,0.449645,0.000000,0.41817,0.562056,CBIO
2022-08-04,0.478570,0.483829,0.468052,0.483829,-0.540094,1407110,1266512.9,0.111011,0.478570,False,False,False,True,0.449645,0.000000,0.41817,0.562056,CBIO
2022-08-05,0.481199,0.491717,0.478570,0.491717,-0.548899,1328388,1230308.4,0.079720,0.483829,False,False,False,True,0.449645,0.000000,0.41817,0.562056,CBIO


In [8]:
stock_df = pd.DataFrame(stock_res, columns = ['Stock','Profit','TradeCount','ProfitTrades','LossTrades'])
df_day = pd.DataFrame(df_data )
df_day.index.name ="Date"

In [9]:
df_day = df_day.loc[df_day['Profit']!=0] # remove 0 values
df_day = df_day.groupby([df_day.index.date]).sum() # групуємо по даті


In [10]:
df_day.sort_index()


,Profit
2022-03-03,500.000000
2022-04-27,-140.000000
2022-05-24,-140.000000
2022-08-09,140.350645


In [11]:
with pd.ExcelWriter("outputFiles/output.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_output.to_excel(writer, sheet_name="full_B")
    df_day.to_excel(writer, sheet_name="by date")
    stock_df.to_excel(writer, sheet_name="statByStock")

In [12]:
#testDF = yf.download("AEI", "2022-10-01", "2022-12-10",back_adjust=True)